#data loading

In [ ]:
import pandas as pd
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers,models
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model

In [ ]:
#Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#reading raw data using pandae dataframe
import pandas as pd
traits=pd.read_csv('/content/drive/MyDrive/DIP Project/test/semi final.xlsx - Form Responses 1.csv')
traits.head()

,Timestamp,Name,University Name,Department ( বিভাগ),Age,hang out,punctuality,assignments,social media,CGPA,skills,study,examination,hard worker,introvert,communication skills,religion,photo
0,10/22/2022 22:32:33,NaN,NaN,NaN,22-26,Yes ( হ্যাঁ ),9,I start two-three days before submission.,3-5 Hour (৩-5 ঘন্টা),No (না),more than 5 Hour ( ৫ ঘন্টার বেশি),3-5 Hour (৩-5 ঘন্টা),No (না),Yes ( হ্যাঁ ),Yes ( হ্যাঁ ),7,7,84013 (1) - Md. Nazmus Sakib.jpg
1,10/22/2022 22:44:06,NaN,NaN,NaN,22-26,Yes ( হ্যাঁ ),6,I start two-three days before submission.,3-5 Hour (৩-5 ঘন্টা),No (না),0-1 Hour (0-১ ঘন্টা),1-3 Hour (১-৩ ঘন্টা),Yes ( হ্যাঁ ),No (না),No (না),9,7,inbound6266109105667462293 - S.M Sifatul Islam...
2,10/22/2022 23:09:45,NaN,NaN,NaN,18-22,Yes ( হ্যাঁ ),7,I start two-three days before submission.,3-5 Hour (৩-5 ঘন্টা),No (না),0-1 Hour (0-১ ঘন্টা),1-3 Hour (১-৩ ঘন্টা),No (না),No (না),No (না),8,9,inbound2206962629156559833 - Md Tanzimul Hasan...
3,10/22/2022 23:10:39,NaN,NaN,NaN,18-22,Yes ( হ্যাঁ ),8,I start toward the end and request sir for ext...,1-3 Hour (১-৩ ঘন্টা),Yes ( হ্যাঁ ),more than 5 Hour ( ৫ ঘন্টার বেশি),1-3 Hour (১-৩ ঘন্টা),Yes ( হ্যাঁ ),No (না),I don't know if I'm an introvert or not. But I...,8,9,Badhon - Prashoman Chakrabarti.jpg
4,10/22/2022 23:20:01,NaN,NaN,NaN,22-26,Yes ( হ্যাঁ ),10,I start toward the end and request sir for ext...,3-5 Hour (৩-5 ঘন্টা),Yes ( হ্যাঁ ),0-1 Hour (0-১ ঘন্টা),1-3 Hour (১-৩ ঘন্টা),Yes ( হ্যাঁ ),Yes ( হ্যাঁ ),I don't know if I'm an introvert or not. But I...,3,9,inbound310068632908722779 - Sajib Talukdar.jpg


#data cleaning

In [ ]:
#Drop unnecessary column
traits.drop(['Name','University Name','Department ( বিভাগ)'],axis=1,inplace=True)

In [ ]:
#replace Column name 
traits=traits.rename({"Age ":"Age"},axis=1)


In [ ]:
traits['introvert']=traits['introvert'].replace("Yes ( হ্যাঁ )",0)\
                                      .replace("No (না)",1)\
                                      .replace("I don't know if I'm an introvert or not. But I cannot talk to strangers easily. But once we get to know each other then we talk a lot.(ইন্ট্রোভার্ট কিনা জানি না।  তবে অপরিচিত কারো সাথে কথা বলতে  পরি না। কিন্তু পরিচয় একবার হয়ে গেলে অনেক বেশি কথা বলি।)",1)
print(traits['introvert'])

0      0
1      1
2      1
3      1
4      1
      ..
97     1
98     1
99     0
100    1
101    0
Name: introvert, Length: 102, dtype: int64


In [ ]:
traits['hang out']=traits['hang out'].replace("Yes ( হ্যাঁ )",1)\
                                      .replace("No (না)",0)\
                                      .replace("I like to chat but now don't get time.(পছন্দ করি কিন্তু আড্ডা দেওয়া হয় নাহ।)",2)
traits['hang out']

0      1
1      1
2      1
3      1
4      1
      ..
97     1
98     1
99     1
100    0
101    1
Name: hang out, Length: 102, dtype: int64

In [ ]:
traits['CGPA']=traits['CGPA'].replace("Yes ( হ্যাঁ )",1)\
                                      .replace("No (না)",0)\
                                     
traits['CGPA']

0      0
1      0
2      0
3      1
4      1
      ..
97     0
98     0
99     0
100    0
101    0
Name: CGPA, Length: 102, dtype: int64

In [ ]:
traits['assignments']=traits['assignments'].replace("I start two-three days before submission.(আমি জমা দেওয়ার দুই-তিন দিন আগে শুরু করি।)",0)\
                                      .replace("I start on very first day. (আমি প্রথম দিন থেকেই শুরু করি। )",1)\
                                      .replace("I start toward the end and request sir for extending the time.(শেষের দিকে শুরু করি এবং স্যারকে সময় বাড়িয়ে দেওয়ার জন্য অনুরোধ করি)",0)\
                                      .replace("I start within two or three days.(আমি দুই-তিন দিনের মধ্যে শুরু করি।)",1)\
                                      .replace("I start two-three days before submission.",0)\
                                      .replace("I start toward the end and request sir for extending the time.",0)\
                                     
traits['assignments']

0      0
1      0
2      0
3      0
4      0
      ..
97     0
98     0
99     1
100    0
101    0
Name: assignments, Length: 102, dtype: int64

In [ ]:
traits['social media']=traits['social media'].replace("0-1 Hour (0-১ ঘন্টা)",1)\
                                      .replace("1-3  Hour (১-৩ ঘন্টা)",1)\
                                      .replace("3-5  Hour (৩-5 ঘন্টা)",0)\
                                      .replace("more than 5 Hour ( ৫ ঘন্টার বেশি)",0)\
                                     
traits['social media']

0      0
1      0
2      0
3      1
4      0
      ..
97     0
98     1
99     1
100    0
101    0
Name: social media, Length: 102, dtype: int64

In [ ]:
traits['skills']=traits['skills'].replace("0-1 Hour (0-১ ঘন্টা)",0)\
                                      .replace("1-3  Hour (১-৩ ঘন্টা)",0)\
                                      .replace("3-5  Hour (৩-5 ঘন্টা)",1)\
                                      .replace("more than 5 Hour ( ৫ ঘন্টার বেশি)",1)\
                                     
traits['skills']

0      1
1      0
2      0
3      1
4      0
      ..
97     0
98     0
99     1
100    0
101    0
Name: skills, Length: 102, dtype: int64

In [ ]:
traits['study']=traits['study'].replace("0-1 Hour (0-১ ঘন্টা)",0)\
                                      .replace("1-3  Hour (১-৩ ঘন্টা)",0)\
                                      .replace("3-5  Hour (৩-5 ঘন্টা)",1)\
                                      .replace("more than 5 Hour ( ৫ ঘন্টার বেশি)",1)\
                                     
traits['study']

0      1
1      0
2      0
3      0
4      0
      ..
97     0
98     0
99     0
100    0
101    0
Name: study, Length: 102, dtype: int64

In [ ]:
traits['examination']=traits['examination'].replace("Yes ( হ্যাঁ )",0)\
                                      .replace("No (না)",1)\
                                      .replace("No, I know that I would have felt the same thing even I had given  one more day.(না, আমি জানি যে আরও একদিন দিলেও আমি একই জিনিস অনুভব করতাম।)",1)
traits['examination']

0      1
1      0
2      1
3      0
4      0
      ..
97     1
98     1
99     1
100    0
101    1
Name: examination, Length: 102, dtype: int64

In [ ]:
traits['hard worker']=traits['hard worker'].replace("Yes ( হ্যাঁ )",1)\
                                      .replace("No (না)",0)
                                     
traits['hard worker']

0      1
1      0
2      0
3      0
4      1
      ..
97     0
98     0
99     1
100    0
101    1
Name: hard worker, Length: 102, dtype: int64

In [ ]:
traits['Age']=traits['Age'].replace("18-22",0)\
                                      .replace("22-26",1)\
                                      .replace("22",1)\
                                      .replace("28",2)\
                                      .replace("32",2)
                                     
traits['Age']

0      1
1      1
2      0
3      0
4      1
      ..
97     1
98     0
99     0
100    0
101    0
Name: Age, Length: 102, dtype: int64

In [ ]:
traits['punctuality']=traits['punctuality'].replace(1,0)\
                                        .replace(2,0)\
                                      .replace(3,0)\
                                      .replace(4,0)\
                                      .replace(5,0)\
                                      .replace(6,1)\
                                      .replace(7,1)\
                                      .replace(8,1)\
                                      .replace(9,1)\
                                      .replace(10,1)
                                     
traits['punctuality']

0      1
1      1
2      1
3      1
4      1
      ..
97     0
98     1
99     1
100    0
101    1
Name: punctuality, Length: 102, dtype: int64

In [ ]:
traits['communication skills']=traits['communication skills'].replace(1,0)\
                                      .replace(2,0)\
                                      .replace(3,0)\
                                      .replace(4,0)\
                                      .replace(5,0)\
                                      .replace(6,1)\
                                      .replace(7,1)\
                                      .replace(8,1)\
                                      .replace(9,1)\
                                      .replace(10,1)
                                     
traits['communication skills']

0      1
1      1
2      1
3      1
4      0
      ..
97     0
98     1
99     1
100    0
101    1
Name: communication skills, Length: 102, dtype: int64

In [ ]:
traits.drop(columns=['religion','Age','hang out'])

,Timestamp,punctuality,assignments,social media,CGPA,skills,study,examination,hard worker,introvert,communication skills,photo
0,10/22/2022 22:32:33,1,0,0,0,1,1,1,1,0,1,84013 (1) - Md. Nazmus Sakib.jpg
1,10/22/2022 22:44:06,1,0,0,0,0,0,0,0,1,1,inbound6266109105667462293 - S.M Sifatul Islam...
2,10/22/2022 23:09:45,1,0,0,0,0,0,1,0,1,1,inbound2206962629156559833 - Md Tanzimul Hasan...
3,10/22/2022 23:10:39,1,0,1,1,1,0,0,0,1,1,Badhon - Prashoman Chakrabarti.jpg
4,10/22/2022 23:20:01,1,0,0,1,0,0,0,1,1,0,inbound310068632908722779 - Sajib Talukdar.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...
97,10/30/2022 17:03:39,0,0,0,0,0,0,1,0,1,0,IMG20221010101739 - Md. Alamin Bhuyan.jpg
98,10/31/2022 9:49:54,1,0,1,0,0,0,1,0,1,1,maom - Md. Abu Obaida Mullick (MAOM).jpg
99,10/31/2022 15:14:42,1,1,1,0,1,0,1,1,0,1,IMG-20221007-WA0196 - abir hossain.jpg
100,11/1/2022 17:45:51,0,0,0,0,0,0,0,0,1,0,IMG_20221101_174219 - Md. Abdur Rahman.jpg


#weight assign and data proccesing

In [ ]:
list_=[]
for i in range(102):
  result=((traits['introvert'][i]*1.5)+(traits['punctuality'][i]*2)+(traits['communication skills'][i]*2)+(traits['hard worker'][i]*2)+(traits['skills'][i]*2.5)+
         (traits['examination'][i]*1)+(traits['study'][i]*1)+(traits['social media'][i]*1)+(traits['assignments'][i]*2)+(traits['CGPA'][i]*1.5))
  list_.append(result)
result

7.0

In [ ]:
class_list=[]
for j in list_:
  if(j < 8):
    class_list.append(0)
  else:
    class_list.append(1)
len(class_list)

102

In [ ]:
filtered=pd.DataFrame({'Selected':class_list,
                   'photo':traits['photo'] })

In [ ]:
filtered.head()

,Selected,photo
0,1,84013 (1) - Md. Nazmus Sakib.jpg
1,0,inbound6266109105667462293 - S.M Sifatul Islam...
2,0,inbound2206962629156559833 - Md Tanzimul Hasan...
3,1,Badhon - Prashoman Chakrabarti.jpg
4,0,inbound310068632908722779 - Sajib Talukdar.jpg


#finding fit for job or not using cnn

In [ ]:
Image_train=[]
Image_test=[]

for i in range(102):
   img=filtered["photo"][i]
   try:
    image_path='/content/drive/MyDrive/image/final segmented image/'+img
    image_r=cv2.imread(image_path)
    image=cv2.resize(image_r,(224,224))
    Class=filtered["Selected"][i]
    Image_train.append([image,Class])
   except:
    print("not done"+str(i))

not done21
not done22
not done34
not done37
not done97


In [ ]:
Image_train=[]
Image_test=[]

for i in range(102):
   img=filtered["photo"][i]
   try:
    image_path='/content/drive/MyDrive/image/final segmented image/'+img
    image_r=cv2.imread(image_path)
    image=cv2.resize(image_r,(224,224))

    if i%5==0:
      Class=filtered["Selected"][i]
      Image_test.append([image,Class])
    else:
      Class=filtered["Selected"][i]
      Image_train.append([image,Class])
   except:
    print("not done"+str(i))

not done21
not done22
not done34
not done37
not done97


In [ ]:
random.shuffle(Image_train)
random.shuffle(Image_test)
X_train=[]
Y_train=[]
X_test=[]
Y_test=[]
for x,y in Image_train:
  X_train.append(x)
  Y_train.append(y)
for x,y in Image_test:
  X_test.append(x)
  Y_test.append(y)

In [ ]:
X_train=np.array(X_train).reshape(-1,224,224,3)
X_test=np.array(X_test).reshape(-1,224,224,3)
X_train=X_train/255.0
X_test=X_test/255
Y_train=np.array(Y_train)
Y_test=np.array(Y_test)

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

In [ ]:
datagen.fit(X_train)

In [ ]:
CNN=models.Sequential([
    layers.Conv2D(128,(3,3),activation='relu',input_shape=(224,224,3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(32,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),
     layers.Conv2D(16,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),
     layers.Conv2D(32,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dense(64,activation='relu'),
    layers.Dense(1,activation='sigmoid')


])
CNN.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 222, 222, 128)     3584      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 111, 111, 128)    0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 109, 109, 64)      73792     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 52, 52, 32)        18464     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 26, 26, 32)      

In [ ]:
CNN.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
CNN.fit(datagen.flow(X_train, Y_train,
         subset='training'),epochs=10)

Epoch 1/10
2/2 [==============================] - 15s 7s/step - loss: 0.6756 - accuracy: 0.6066
Epoch 2/10
2/2 [==============================] - 14s 7s/step - loss: 0.6936 - accuracy: 0.6066
Epoch 3/10
2/2 [==============================] - 18s 10s/step - loss: 0.6775 - accuracy: 0.5902
Epoch 4/10
2/2 [==============================] - 14s 7s/step - loss: 0.6740 - accuracy: 0.6066
Epoch 5/10
2/2 [==============================] - 14s 7s/step - loss: 0.6983 - accuracy: 0.6066
Epoch 6/10
2/2 [==============================] - 14s 7s/step - loss: 0.6720 - accuracy: 0.6066
Epoch 7/10
2/2 [==============================] - 14s 7s/step - loss: 0.6682 - accuracy: 0.6066
Epoch 8/10
2/2 [==============================] - 14s 6s/step - loss: 0.6654 - accuracy: 0.6066
Epoch 9/10
2/2 [==============================] - 14s 6s/step - loss: 0.6762 - accuracy: 0.6066
Epoch 10/10
2/2 [==============================] - 14s 7s/step - loss: 0.6843 - accuracy: 0.6066


In [ ]:
CNN.evaluate(X_test,Y_test)

1/1 [==============================] - 1s 1s/step - loss: 0.7232 - accuracy: 0.4286


[0.7231748104095459, 0.4285714328289032]

#finding fit for job or not VGG19

In [ ]:
Image_train=[]
Image_test=[]

for i in range(102):
   img=filtered["photo"][i]
   try:
    image_path='/content/drive/MyDrive/image/final segmented image/'+img
    image_r=cv2.imread(image_path)
    image=cv2.resize(image_r,(224,224))

    if i%5==0:
      Class=filtered["Selected"][i]
      Image_test.append([image,Class])
    else:
      Class=filtered["Selected"][i]
      Image_train.append([image,Class])
   except:
    print("not done"+str(i))

not done21
not done22
not done34
not done37
not done97


In [ ]:
random.shuffle(Image_train)
random.shuffle(Image_test)
X_train=[]
Y_train=[]
X_test=[]
Y_test=[]
for x,y in Image_train:
  X_train.append(x)
  Y_train.append(y)
for x,y in Image_test:
  X_test.append(x)
  Y_test.append(y)

In [ ]:
X_train=np.array(X_train).reshape(-1,224,224,3)
X_test=np.array(X_test).reshape(-1,224,224,3)
X_train=X_train/255.0
X_test=X_test/255
Y_train=np.array(Y_train)
Y_test=np.array(Y_test)

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

In [ ]:
datagen.fit(X_train)

In [ ]:
from keras.applications.vgg19 import VGG19,preprocess_input

In [ ]:
image_size=[224,224]
Vgg=VGG19(input_shape=image_size + [3],weights='imagenet',include_top=False)
for layer in Vgg.layers:
  layer.trainable=False

80134624/80134624 [==============================] - 0s 0us/step


In [ ]:
F=layers.Flatten()(Vgg.output)
prediction=layers.Dense(1,activation='sigmoid')(F)

In [ ]:
VGG=Model(inputs=Vgg.input,outputs=prediction)
VGG.summary

<bound method Model.summary of <keras.engine.functional.Functional object at 0x7ff07937bc10>>

In [ ]:
VGG.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
VGG.fit(datagen.flow(X_train, Y_train,
         subset='training'),epochs=10)

Epoch 1/10
2/2 [==============================] - 39s 18s/step - loss: 0.7321 - accuracy: 0.5246
Epoch 2/10
2/2 [==============================] - 36s 17s/step - loss: 0.7270 - accuracy: 0.5574
Epoch 3/10
2/2 [==============================] - 36s 19s/step - loss: 0.6536 - accuracy: 0.6066
Epoch 4/10
2/2 [==============================] - 35s 18s/step - loss: 0.5912 - accuracy: 0.6885
Epoch 5/10
2/2 [==============================] - 36s 17s/step - loss: 0.5520 - accuracy: 0.7049
Epoch 6/10
2/2 [==============================] - 38s 19s/step - loss: 0.5483 - accuracy: 0.7377
Epoch 7/10
2/2 [==============================] - 35s 18s/step - loss: 0.5089 - accuracy: 0.8033
Epoch 8/10
2/2 [==============================] - 36s 17s/step - loss: 0.5129 - accuracy: 0.7705
Epoch 9/10
2/2 [==============================] - 36s 17s/step - loss: 0.4504 - accuracy: 0.8033
Epoch 10/10
2/2 [==============================] - 37s 20s/step - loss: 0.4512 - accuracy: 0.8033


In [ ]:
VGG.evaluate(X_test,Y_test)

1/1 [==============================] - 12s 12s/step - loss: 0.7925 - accuracy: 0.5714


[0.7924944758415222, 0.5714285969734192]

#finding fit for job or not using VGG19 without data agmenteyion

In [ ]:
Image_train=[]
Image_test=[]

for i in range(102):
   img=filtered["photo"][i]
   try:
    image_path='/content/drive/MyDrive/image/final segmented image/'+img
    image_r=cv2.imread(image_path)
    image=cv2.resize(image_r,(224,224))

    if i%5==0:
      Class=filtered["Selected"][i]
      Image_test.append([image,Class])
    else:
      Class=filtered["Selected"][i]
      Image_train.append([image,Class])
   except:
    print("not done"+str(i))

not done21
not done22
not done34
not done37
not done97


In [ ]:
random.shuffle(Image_train)
random.shuffle(Image_test)
X_train=[]
Y_train=[]
X_test=[]
Y_test=[]
for x,y in Image_train:
  X_train.append(x)
  Y_train.append(y)
for x,y in Image_test:
  X_test.append(x)
  Y_test.append(y)

In [ ]:
X_train=np.array(X_train).reshape(-1,224,224,3)
X_test=np.array(X_test).reshape(-1,224,224,3)
X_train=X_train/255.0
X_test=X_test/255
Y_train=np.array(Y_train)
Y_test=np.array(Y_test)

In [ ]:
from keras.applications.vgg19 import VGG19,preprocess_input
image_size=[224,224]
Vgg=VGG19(input_shape=image_size + [3],weights='imagenet',include_top=False)
for layer in Vgg.layers:
  layer.trainable=False

In [ ]:
F=layers.Flatten()(Vgg.output)
prediction=layers.Dense(1,activation='sigmoid')(F)

In [ ]:
VGG=Model(inputs=Vgg.input,outputs=prediction)

In [ ]:
VGG.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
VGG.fit(X_train,Y_train,epochs=10)

Epoch 1/10
3/3 [==============================] - 46s 13s/step - loss: 1.2700 - accuracy: 0.5000
Epoch 2/10
3/3 [==============================] - 47s 13s/step - loss: 0.8410 - accuracy: 0.5132
Epoch 3/10
3/3 [==============================] - 43s 13s/step - loss: 0.5889 - accuracy: 0.6579
Epoch 4/10
3/3 [==============================] - 43s 13s/step - loss: 0.5320 - accuracy: 0.6711
Epoch 5/10
3/3 [==============================] - 45s 14s/step - loss: 0.5887 - accuracy: 0.5658
Epoch 6/10
3/3 [==============================] - 43s 13s/step - loss: 0.3481 - accuracy: 0.8947
Epoch 7/10
3/3 [==============================] - 43s 13s/step - loss: 0.4027 - accuracy: 0.7632
Epoch 8/10
3/3 [==============================] - 43s 13s/step - loss: 0.2895 - accuracy: 0.9211
Epoch 9/10
3/3 [==============================] - 45s 13s/step - loss: 0.3169 - accuracy: 0.8947
Epoch 10/10
3/3 [==============================] - 43s 13s/step - loss: 0.2388 - accuracy: 0.9079


In [ ]:
VGG.evaluate(X_test,Y_test)

1/1 [==============================] - 12s 12s/step - loss: 1.0153 - accuracy: 0.5714


[1.0153024196624756, 0.5714285969734192]